In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.naive_bayes import BernoulliNB
%matplotlib inline

In [2]:
# load the data

df = pd.read_csv('RegularSeasonDetailedResults.csv')

In [3]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [4]:
# create training set from this data

training_set = pd.DataFrame()

In [5]:
training_set['net_fgm'] = df['WFGM'] - df['LFGM']
training_set['net_fga'] = df['WFGA'] - df['LFGA']
training_set['net_fgm3'] = df['WFGM3'] - df['LFGM3']
training_set['net_fga3'] = df['WFGA3'] - df['LFGA3']
training_set['net_ftm'] = df['WFTM'] - df['LFTM']
training_set['net_fta'] = df['WFTA'] - df['LFTA']
training_set['net_or'] = df['WOR'] - df['LOR']
training_set['net_dr'] = df['WDR'] - df['LDR']
training_set['net_tr'] = df['WOR'] + df['WDR'] - df['LOR'] - df['LDR']
training_set['net_ast'] = df['WAst'] - df['LAst']
training_set['net_to'] = df['WTO'] - df['LTO']
training_set['net_stl'] = df['WStl'] - df['LStl']
training_set['net_blk'] = df['WBlk'] - df['LBlk']
training_set['net_pf'] = df['WPF'] - df['LPF']
training_set['win'] = 1

In [6]:
inverse_df = -training_set
inverse_df['win'] = 0

In [7]:
# this is the final version of the training set
# X_train = all columns except 'win'
# Y_train = win column

final_df = training_set.append(inverse_df)

Next, create the test set. This competition will be graded solely on the 2019 NCAA tournament, so the data used to create the test set will come solely from games played in the 2018 season (in collegiate sports, the change in year-over-year performance can be drastic due to the constant roster changes).

The objective here is to find each team's average in each metric provided in the training set (average net_fgm, average net_fga, etc.). Then, for any given matchup of team A vs. team B, compare their averages in each metric - the differences in these averages will yield the final data to be evaluated as the test set!

In [8]:
df_2018 = df.loc[df['Season'] == 2018]

In [9]:
df_2018_net = pd.DataFrame()

In [10]:
df_2018_net['teamid'] = df_2018['WTeamID']
df_2018_net['net_fgm'] = df_2018['WFGM'] - df_2018['LFGM']
df_2018_net['net_fga'] = df_2018['WFGA'] - df_2018['LFGA']
df_2018_net['net_fgm3'] = df_2018['WFGM3'] - df_2018['LFGM3']
df_2018_net['net_fga3'] = df_2018['WFGA3'] - df_2018['LFGA3']
df_2018_net['net_ftm'] = df_2018['WFTM'] - df_2018['LFTM']
df_2018_net['net_fta'] = df_2018['WFTA'] - df_2018['LFTA']
df_2018_net['net_or'] = df_2018['WOR'] - df_2018['LOR']
df_2018_net['net_dr'] = df_2018['WDR'] - df_2018['LDR']
df_2018_net['net_tr'] = df_2018['WOR'] + df_2018['WDR'] - df_2018['LOR'] - df_2018['LDR']
df_2018_net['net_ast'] = df_2018['WAst'] - df_2018['LAst']
df_2018_net['net_to'] = df_2018['WTO'] - df_2018['LTO']
df_2018_net['net_stl'] = df_2018['WStl'] - df_2018['LStl']
df_2018_net['net_blk'] = df_2018['WBlk'] - df_2018['LBlk']
df_2018_net['net_pf'] = df_2018['WPF'] - df_2018['LPF']

In [11]:
df_2018_inverse = -df_2018_net
df_2018_inverse['teamid'] = df_2018['LTeamID']

In [12]:
aggregate_2018_df = df_2018_net.append(df_2018_inverse)

In [13]:
test_data = aggregate_2018_df.groupby('teamid')['net_fgm', 'net_fga', 'net_fgm3', 'net_fga3', 'net_ftm', 'net_fta', 'net_or',
                                             'net_dr', 'net_tr', 'net_ast', 'net_to', 'net_stl', 'net_blk', 'net_pf'].mean()

In [14]:
test_data.head()

,net_fgm,net_fga,net_fgm3,net_fga3,net_ftm,net_fta,net_or,net_dr,net_tr,net_ast,net_to,net_stl,net_blk,net_pf
teamid,,,,,,,,,,,,,,
1101,0.740741,2.185185,-0.148148,0.222222,-3.481481,-4.777778,-0.888889,-1.814815,-2.703704,1.851852,-1.074074,0.629630,0.703704,3.370370
1102,-2.482759,1.655172,-1.344828,1.068966,0.275862,-0.206897,0.655172,-3.344828,-2.689655,-0.689655,-1.206897,0.862069,-0.965517,-0.586207
1103,-1.451613,1.193548,1.290323,4.032258,-3.290323,-3.870968,0.258065,-1.935484,-1.677419,-0.419355,0.838710,-0.419355,-1.258065,2.645161
1104,1.117647,-3.352941,-0.558824,-2.058824,0.705882,1.323529,-1.558824,1.764706,0.205882,0.911765,0.970588,-0.176471,1.705882,-0.970588
1105,-6.709677,-4.741935,-1.483871,0.193548,-1.032258,-0.516129,-0.258065,-2.322581,-2.580645,-3.870968,5.193548,-3.677419,-3.290323,-0.741935


In [15]:
submission_df = pd.read_csv('SampleSubmissionStage2.csv')

In [16]:
# create final dataframe to submit into the models for prediction

test_df = pd.DataFrame()
test_df['team1'] = submission_df['ID'].str[5:9]
test_df['team2'] = submission_df['ID'].str[10:14]

In [17]:
test_df.head()
# test_df['extra columns'] = test_data.loc['team1'] - test_data.loc['team2']

,team1,team2
0,1101,1113
1,1101,1120
2,1101,1124
3,1101,1125
4,1101,1133


In [18]:
bnb = BernoulliNB()

x_train = final_df.drop(columns = 'win')
y_train = final_df['win']
bnb.fit(x_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [19]:
dummy_test = bnb.predict(test_data)

In [20]:
print(dummy_test)

[0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0 0
 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 0 0
 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 1 0 1 1 0 1 1 0 1 1
 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1
 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0
 1 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 1 1 1 1 0 1 1
 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1
 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 0]
